In [ ]:
!pip install scnym
!pip install anndata==0.8.0

In [ ]:
import pandas as pd
import anndata as ad
import scanpy as sc
from itertools import combinations
import numpy as np
import scnym
from sklearn.metrics import balanced_accuracy_score
import warnings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Unsupervised training for scNym

In [ ]:
def run_unsupervised(PATH_in, PATH_model, PATH_out):
  adata = ad.read_h5ad(PATH_in)
  adata.obs["label"][adata.obs["domain"]==1] = -9
  CONFIGS["no_new_identity"]["weighted_class"]=True
  scnym.api.UNLABELED_TOKEN = -9
  scnym.api.scnym_api(
    adata=adata,
    task="train",
    groupby="label",
    domain_groupby="domain",
    config = "no_new_identity",
    out_path=PATH_model
  )
  scnym.api.scnym_api(
    adata=adata,
    task='predict',
    groupby="label",
    domain_groupby="domain",
    trained_model=PATH_model,
  )
  df = pd.DataFrame.from_dict({"y_true" : adata.obs["cell_type"][adata.obs["domain"]==1],
                               "y_pred" : adata.obs["scNym"][adata.obs["domain"]==1]})
  df.to_csv(PATH_out)
  return df

In [ ]:
df_bm = run_unsupervised("/content/drive/MyDrive/data/bm_for_scnym.h5ad",
                         "/content/drive/MyDrive/data/scnym_bm",
                         "/content/drive/MyDrive/data/bm_scnym_all.csv")
df_pancreas = run_unsupervised("/content/drive/MyDrive/data/pancreas_for_scnym.h5ad",
                         "/content/drive/MyDrive/data/scnym_pancreas",
                         "/content/drive/MyDrive/data/pancreas_scnym_all.csv")
df_brain = run_unsupervised("/content/drive/MyDrive/data/brain_for_scnym.h5ad",
                         "/content/drive/MyDrive/data/scnym_brain",
                         "/content/drive/MyDrive/data/brain_scnym_all.csv")

In [ ]:
def run_unsupervised_rev(PATH_in, PATH_model, PATH_out):
  adata = ad.read_h5ad(PATH_in)
  adata.obs["label"][adata.obs["domain"]==0] = -9
  CONFIGS["no_new_identity"]["weighted_class"]=True
  scnym.api.UNLABELED_TOKEN = -9
  scnym.api.scnym_api(
    adata=adata,
    task="train",
    groupby="label",
    domain_groupby="domain",
    config = "no_new_identity",
    out_path=PATH_model
  )
  scnym.api.scnym_api(
    adata=adata,
    task='predict',
    groupby="label",
    domain_groupby="domain",
    trained_model=PATH_model,
  )
  df = pd.DataFrame.from_dict({"y_true" : adata.obs["cell_type"][adata.obs["domain"]==0],
                               "y_pred" : adata.obs["scNym"][adata.obs["domain"]==0]})
  df.to_csv(PATH_out)
  return df

In [ ]:
df_bm = run_unsupervised_rev("/content/drive/MyDrive/data/bm_for_scnym.h5ad",
                         "/content/drive/MyDrive/data/scnym_bm",
                         "/content/drive/MyDrive/data/bm_scnym_pred_rev.csv")
df_pancreas = run_unsupervised_rev("/content/drive/MyDrive/data/pancreas_for_scnym.h5ad",
                         "/content/drive/MyDrive/data/scnym_pancreas",
                         "/content/drive/MyDrive/data/pancreas_scnym_pred_rev.csv")
df_brain = run_unsupervised_rev("/content/drive/MyDrive/data/brain_for_scnym.h5ad",
                         "/content/drive/MyDrive/data/scnym_brain",
                         "/content/drive/MyDrive/data/brain_scnym_pred_rev.csv")